In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")


In [ ]:
df.columns=[x.lower() for x in df.columns]
df.corr()

1. Removing OUTLIERS

In [ ]:
nomv=df.dropna(axis=0)
#nomv["seller_type"]=nomv["seller_type"].map({"Individual":0,"Dealer":1})
#nomv["fuel_type"] =nomv['fuel_type'].map({"Petrol":1,"Diesel":0})
#nomv["transmission"]=nomv["transmission"].map({"Manual":1,"Automatic":1})
sns.distplot(nomv["selling_price"])
q=nomv["selling_price"].quantile(0.999)
dta1=nomv[nomv["selling_price"]<q]


In [ ]:
q2=dta1["present_price"].quantile(0.99)
q3=dta1["present_price"].quantile(0.1)
dta2=dta1[dta1["present_price"]<q2]
dta3=dta2[dta2["present_price"]>q3]
sns.distplot(dta3["present_price"])

In [ ]:
sns.distplot(df["kms_driven"])

In [ ]:
q4=dta3["kms_driven"].quantile(0.99)
dta4=dta3[dta3["kms_driven"]<q4]
sns.distplot(dta4["kms_driven"])


In [ ]:
cleaned=dta4
x=cleaned.drop(["selling_price","car_name","year"],axis=1)
y=cleaned["selling_price"]
xdummies=pd.get_dummies(x,drop_first=True)
xdummies["year"]=cleaned["year"]
cleaned["selling_price"]=np.log(cleaned["selling_price"])


1. Homoscedasity

In [ ]:
fig,(ax1,ax2)=plt.subplots(nrows=1,ncols=2,figsize=(4,4))
ax1.scatter(cleaned["selling_price"],cleaned["present_price"],label="yo")
ax1.legend()
ax2.scatter(cleaned["selling_price"],cleaned["kms_driven"])
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
scaler=StandardScaler().fit(xdummies)
ind=scaler.fit(xdummies).transform(xdummies)
xtrain,xtest,ytrain,ytest=train_test_split(ind,y,test_size=0.2,shuffle=False,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

1. **Building linear regression**

In [ ]:
reg=LinearRegression()
mymodel=reg.fit(xtrain,ytrain)

1. Statistics of my model

In [ ]:
coefficeints=mymodel.coef_
intercept=mymodel.intercept_
r_squared=mymodel.score(xtrain,ytrain)
p_values=f_regression(xtrain,ytrain)[1].round(3)
adj_r2=1-(1-r_squared)*(173-1)/172
lst={"coefficeints":coefficeints,"intercept":intercept,"r_squared":r_squared,"pvalues":p_values}
lst

1. plotting selling_price vs predicted

In [ ]:
pred=mymodel.predict(xtrain)
plt.scatter(pred,ytrain)

In [ ]:
pd.options.display.max_rows=900
ccc=pd.DataFrame({"p":pred,"y":ytrain,"difference":pred-ytrain})
ccc.describe(include="all")

1. Plotting (Testing selling_price vs predicted selling_price)

In [ ]:
tpred=mymodel.predict(xtest)
plt.scatter(ytest,tpred)

1. HERE IS MY MODEL,HERE YOU CAN PREDICT selling_price by mymodel.predict(ENTER YOUR VALUES HERE)

In [ ]:
features=nomv.drop(["selling_price","car_name"],axis=1)
new_xdummies=pd.get_dummies(features,drop_first=True)
new_ind=scaler.fit(new_xdummies).transform(new_xdummies)
new_pred=mymodel.predict(new_ind)
final_submit=pd.DataFrame(np.exp(new_pred),columns=["Predicted selling_price"])


In [ ]:
final_submit

In [ ]:
final_submit.to_csv("Submission.csv")

# Accuracy Scores and Statistics of my model

In [ ]:
lst